In [84]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [85]:
data_loc= "/home/shubham/Desktop/Function_App_Testing/HttpTrigger1/Guest Recommender.csv"

df=pd.read_csv(data_loc)

pd.set_option('display.max_columns', None)

df['ARRDAT'] = pd.to_datetime(df['ARRDAT'], format='%Y%m%d')

df['DEPDAT'] = pd.to_datetime(df['DEPDAT'], format='%Y%m%d')

df['RMNTS']=df['DEPDAT']-df['ARRDAT']

df['RMNTS']=df['RMNTS'].astype('timedelta64[D]')

df.drop(['REGNUB','FOLNUB','ROMNUB','ARRDAT','DEPDAT','TRC','COM','CHQ','TRD','CAS','ADQ','CRD','ADC','BOH','ADV','POT'],axis=1,inplace=True)

df[['TRS', 'PHT', 'SPC', 'BBD', 'GBR', 'JVH', 'MIT', 'EP', 'VCH', 'RNT', 'ITV', 'CCF', 'PHC', 'TIP', 'EXB', 'STD', 'ART', 'SEC', 'NOT', 'FAX', 'FST', 'BB', 'RMS', 'ITB', 'HTM', 'TRF', 'RTN', 'LCA', 'PLT', 'PLC', 'BTB', 'FNC', 'SSG', 'LLT', 'LAU', 'IDD', 'JBR', 'MIS', 'AP', 'FX']]=df[['TRS', 'PHT', 'SPC', 'BBD', 'GBR', 'JVH', 'MIT', 'EP', 'VCH', 'RNT', 'ITV', 'CCF', 'PHC', 'TIP', 'EXB', 'STD', 'ART', 'SEC', 'NOT', 'FAX', 'FST', 'BB', 'RMS', 'ITB', 'HTM', 'TRF', 'RTN', 'LCA', 'PLT', 'PLC', 'BTB', 'FNC', 'SSG', 'LLT', 'LAU', 'IDD', 'JBR', 'MIS', 'AP', 'FX']].div(df.RMNTS, axis=0)

df.loc[df['RMNTS']==0]

df.isnull().sum() # This is to confirm that division by 0 has caused the NaN

df.dropna(axis=0,how='any', inplace=True) # if any value in the row is NaN, it will be removed. Else use how='all'

df.drop(['SPC'],axis=1,inplace=True)

df['BBD'].describe()

df.drop(['BBD'],axis=1,inplace=True)


df.drop(['GBR'],axis=1,inplace=True)

# #### Find all columns of the dataframe that have all 0 values in it. (It would be better to drop these in one go, than one by one).

zeros=df.loc[:,(df==0).all()] # 17 columns of the dataframe are fully 0s

df.drop(columns=zeros,axis=1,inplace=True) # zeros was assigned columns of the df in the previous cell


mms=MinMaxScaler([0,5]) # The parameter passed the range of values min=0 and max=5

df[['PHT','JVH','MIT','EP','VCH','EXB','ART','SEC','FAX','BB','RMS','ITB','HTM','TRF','RTN','BTB','FNC','LAU','MIS','RMNTS']]=mms.fit_transform(df[['PHT','JVH','MIT','EP','VCH','EXB','ART','SEC','FAX','BB','RMS','ITB','HTM','TRF','RTN','BTB','FNC','LAU','MIS','RMNTS']])

df=df.round(decimals=2)

df=df.drop(['GSTNAM'], axis=1) #Dropping Guest names so as to avoid confusions as there are several guests with same name


In [86]:
df['RMNTS'].dtypes

dtype('float64')

In [7]:
gstcod = 16598

In [8]:
df[df['GSTCOD']==gstcod]
#Grouping by guest code and returning the mean of rest of the columns
df1 = df.groupby(['GSTCOD']).mean()
df1.reset_index(inplace=True)
df2 = df1.melt(id_vars=['GSTCOD'],var_name='Services',value_name='Rating')
inputuser = df2[df2['GSTCOD']==gstcod]

In [9]:
inputuser.head()

,GSTCOD,Services,Rating
319,16598,PHT,0.0
4122,16598,JVH,0.0
7925,16598,MIT,0.0
11728,16598,EP,0.0
15531,16598,VCH,0.0


In [12]:
df1.head()

,GSTCOD,PHT,JVH,MIT,EP,VCH,EXB,ART,SEC,FAX,BB,RMS,ITB,HTM,TRF,RTN,BTB,FNC,LAU,MIS,RMNTS
0,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.88,0.0,0.30,0.0,0.0,0.0,0.00
1,84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.56,0.0,0.00,0.0,0.0,0.0,0.15
2,109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.43,0.0,0.18,0.0,0.0,0.0,0.00
3,136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.88,0.0,0.00,0.0,0.0,0.0,0.07
4,234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.43,0.0,0.00,0.0,0.0,0.0,0.22


In [13]:
df2.head()

,GSTCOD,Services,Rating
0,59,PHT,0.0
1,84,PHT,0.0
2,109,PHT,0.0
3,136,PHT,0.0
4,234,PHT,0.0


In [37]:
from azure.datalake.store import core, lib, multithread
import pandas as pd

In [38]:
token = lib.auth('9be10acf-2a10-42a0-9748-00a93cd07042', 'shivangg@dynapt.ai', '11QQqq!!')
adl = core.AzureDLFileSystem(token, store_name='recomdata')

In [39]:
with adl.open('demo/Guest Recommender.csv', blocksize=2**20) as f:
    print(f.readline())
    print(f.readline())
    print(f.readline())
    # could have passed f to any function requiring a file object:
    # pandas.read_csv(f)

DatalakeRESTException: HTTP error: ConnectionError(MaxRetryError("HTTPSConnectionPool(host='recomdata.azuredatalakestore.net', port=443): Max retries exceeded with url: /webhdfs/v1/demo/Guest%20Recommender.csv?OP=GETFILESTATUS&api-version=2018-09-01 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fa42b6a6748>: Failed to establish a new connection: [Errno -2] Name or service not known',))",),)

In [47]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
import tables

STORAGEACCOUNTNAME= 'recommendtest'
STORAGEACCOUNTKEY= 'eZ6rUu9dIV3I0ZZgyXDm2yAe+dJJ8m7C3YTlMuOUqHD7EWck9hLFVEamxJa9RQgIty81t32zNPjUie9Mt4rd9Q=='
CONTAINERNAME= 'demo'
BLOBNAME= 'Guest Recommender.csv'

#download from blob

blob_service=BlobServiceClient.from_connection_string('DefaultEndpointsProtocol=https;AccountName=recommendtest;AccountKey=eZ6rUu9dIV3I0ZZgyXDm2yAe+dJJ8m7C3YTlMuOUqHD7EWck9hLFVEamxJa9RQgIty81t32zNPjUie9Mt4rd9Q==;EndpointSuffix=core.windows.net', container_name="demo", blob_name="Guest Recommender.csv")

In [49]:
blob_data = blob_service.download_blob()

AttributeError: 'BlobServiceClient' object has no attribute 'download_blob'

In [81]:
from azure.storage.blob import BlobClient

blob = BlobClient(account_url="https://recommendtest.blob.core.windows.net/",
                  container_name="demo",
                  blob_name="Guest Recommender.csv",
                  credential="eZ6rUu9dIV3I0ZZgyXDm2yAe+dJJ8m7C3YTlMuOUqHD7EWck9hLFVEamxJa9RQgIty81t32zNPjUie9Mt4rd9Q==")

# with open("example.csv", "wb") as f:
data = blob.download_blob()
# data.readinto(f)

In [83]:
df = pd.DataFrame([x.replace('\r', '').split(',') for x in data.content_as_text().split('\n')])
df.columns = df.iloc[0]
df = df.drop(0)
df = df.reset_index(drop=True)
df.head()

,REGNUB,FOLNUB,ROMNUB,GSTCOD,GSTNAM,ARRDAT,DEPDAT,TRS,PHT,SPC,BBD,TRC,COM,GBR,JVH,CHQ,TRD,MIT,EP,VCH,RNT,ITV,CCF,PHC,TIP,EXB,STD,ART,SEC,NOT,FAX,CAS,FST,BB,ADQ,RMS,ITB,HTM,CRD,ADC,TRF,RTN,LCA,PLT,PLC,BTB,FNC,SSG,BOH,LLT,LAU,IDD,JBR,ADV,MIS,AP,POT,FX
0,19547,1,1,16598,MR TUIDAM ROMA,20171230,20180101,0,0,0,0,0,-2970,0,0,0,990,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19573,1,9,5238,MR BANIYAMAI ALLAN,20180105,20180114,0,221.82,0,0,-4535,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3384.55,0,0,0,0,568,0,0,0,0,0,0,0,0,0,0,0,0
2,19579,1,99,16617,LTD PUMA ENERGY,20180106,20180106,0,0,0,0,0,-560,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,562,0,0,0,0,0,0,0,-2,0,0,0,0
3,19631,1,12,16679,MR LUCAS KINDIWA,20180116,20180120,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,54.54,0,0,0,0,0,0,-1692,0,0,0,0,0,1483.64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19638,1,14,16673,MR MAINO ANDREW,20180118,20180120,0,0,0,0,-889,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-22,0,0,0,0,0,0,0,0,788.18,0,0,0,0,44,0,0,0,0,0,0,0,0,0,0,0,0


In [80]:
df['FX']

KeyError: 'FX'